In [1]:
import os
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from scipy.stats import entropy, kurtosis
import xgboost as xgb
import seaborn as sns
from xgboost import plot_importance
import warnings
import matplotlib.pyplot as plt
import pandas as pd
from math import *
import numpy as np

pd.set_option('display.max_columns', None)
from IPython.display import display
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
pathf = os.path.join("..", "data", "particles")

trainpd = pd.read_csv(os.path.join(pathf, "train.csv"))
print(trainpd.head(1))
trainshape = trainpd.shape
print(trainshape)
eventpd = pd.read_csv(os.path.join(pathf, "event.csv"))
print(eventpd.head(1))
print(eventpd.shape)
testpd = pd.read_csv(os.path.join(pathf, "test.csv"))
testshape = testpd.shape
print(testpd.head(1))
print(testpd.shape)

data = pd.concat([trainpd, testpd], ignore_index=True)
data = pd.merge(data, eventpd, on='event_id', how='left')

       x      y  z        t   terror        q  flag  event_id  hit_id
0 -142.5 -147.5  0  767.879  2.02966  1.05052     0         7       1
(9473201, 9)
   event_id  nhit  nhitreal  energymc  thetamc    phimc   xcmc    ycmc
0         7   426        70   48348.9  63.1686  11.0982 -40.83  114.03
(13315, 8)
       x      y  z        t  terror        q  event_id  hit_id
0 -142.5 -127.5  0  848.061  1.9984  1.15067         9       1
(4086511, 8)


In [3]:
# (k(q,mc)*t0)^2 + dis^2 -dis*cos(phi)*sin(thmc)*t0 = t^2
data['fx'] = data['x'] - data['xcmc']
data['fy'] = data['y'] - data['ycmc']
data['fphi'] = np.arctan2(data['fy'], data['fx'])-(data['phimc'] * np.pi / 180.)
data['fdis'] = np.sqrt(data['fx'] ** 2 + data['fy'] ** 2)
data['fsinthmc'] = np.sin(data['thetamc'] * np.pi / 180.)
data['fcosphi'] = np.cos(data['fphi'] * np.pi / 180.)

data['ft2'] = data['t'] ** 2
data['fdis2'] = data['fdis'] ** 2 
data['fsencond'] = data['fdis'] * data['fcosphi'] * data['fsinthmc']

data['fttrue'] = data['t'] / data['terror']
data['nhitratio'] = data['nhit'] / data['nhitreal']

# del data['fx']
# del data['fy']
del data['x']
del data['y']
del data['z']

In [4]:
info_new = pd.DataFrame()
info_new["event_id"] = data.groupby(["event_id"])["event_id"].mean()
info_new["fdis_mean"] = data.groupby(["event_id"])["fdis"].mean()
info_new["fdis_std"] = data.groupby(["event_id"])["fdis"].std()
info_new["fdis_stdmean"] = info_new["fdis_std"] / info_new["fdis_mean"]
info_new["ft_mean"] = data.groupby(["event_id"])["t"].mean()
info_new["ft_std"] = data.groupby(["event_id"])["t"].std()
info_new["ft_stdmean"] = info_new["ft_std"] / info_new["ft_mean"]
info_new["ft_mean2"] = info_new["ft_mean"] ** 2
info_new.reset_index(drop=True, inplace=True)
data = pd.merge(data, info_new, on='event_id', how='left')

In [5]:
print(trainshape[0])
print(type(trainshape[0]))
print(data.shape)

trainpd = data[:trainshape[0]].reset_index()
testpd = data[trainshape[0]:].reset_index()
del data

9473201
<class 'int'>
(13559712, 27)


In [6]:
print(trainpd.columns)
feature= [x for x in trainpd.columns if x not in ['flag','index','hit_id','event_id']]
labels = trainpd['flag']
del trainpd['flag']
del testpd['flag']

Index(['index', 'event_id', 'flag', 'hit_id', 'q', 't', 'terror', 'nhit',
       'nhitreal', 'energymc', 'thetamc', 'phimc', 'xcmc', 'ycmc', 'fx', 'fy',
       'fphi', 'fdis', 'fsinth', 'fcosphi', 'fttrue', 'nhitratio', 'fdis_mean',
       'fdis_std', 'fdis_stdmean', 'ft_mean', 'ft_std', 'ft_stdmean'],
      dtype='object')


In [ ]:
#这里用的gpu版的xgb，cpu的话实在太慢了，建议装个gpu版
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=4399)
y_pp_xgb = np.zeros(testshape[0])
y_pp_xgb_stacking = np.zeros(len(labels))
for train_index, test_index in kf.split(trainpd):
    print ( ">>>", train_index )
    clf = xgb.XGBClassifier(tree_method='gpu_hist',max_depth=8,learning_rate=0.1,verbosity=3,
                           eval_metric='auc',n_estimators=2000)
    clf.fit(
        trainpd[feature].iloc[train_index], labels[train_index],
        eval_set=[(trainpd[feature].iloc[train_index], labels[train_index]),
                  (trainpd[feature].iloc[test_index], labels[test_index])],
        early_stopping_rounds=50,
        verbose=True,
    )

    y_pred = clf.predict(trainpd[feature].iloc[test_index]) 
    y_predprob = clf.predict_proba(trainpd[feature].iloc[test_index])[:, 1] 
    
    y_pp_xgb_stacking[test_index] = y_predprob
      
    auc = metrics.roc_auc_score(labels[test_index], y_predprob)
    print("AUC Score (Train): %f" % auc) 
    
    y_pp_xgb += clf.predict_proba(testpd[feature])[:, 1] / n_splits

>>> [      0       2       3 ... 9473198 9473199 9473200]
[18:46:12] DEBUG: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/tree/updater_gpu_hist.cu:1167: [GPU Hist]: Configure
[18:46:16] ======== Monitor: xgboost::common::DenseCuts::DenseCuts ========
[18:46:16] Init: 0.010708s, 1 calls @ 10708us

[18:46:17] ======== Monitor: xgboost::common::HistogramCuts::HistogramCuts ========
[0]	validation_0-auc:0.96706	validation_1-auc:0.96699
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 50 rounds.
[1]	validation_0-auc:0.97024	validation_1-auc:0.97019
[2]	validation_0-auc:0.97178	validation_1-auc:0.97167
[3]	validation_0-auc:0.97276	validation_1-auc:0.97263
[4]	validation_0-auc:0.97353	validation_1-auc:0.97340
[5]	validation_0-auc:0.97475	validation_1-auc:0.97460
[6]	validation_0-auc:0.97524	validation_1-auc:0.97508
[7]	validation_0-auc:0.97561	validation_1-auc:0.97544
[8]	valida

[139]	validation_0-auc:0.99359	validation_1-auc:0.99337
[140]	validation_0-auc:0.99372	validation_1-auc:0.99349
[141]	validation_0-auc:0.99373	validation_1-auc:0.99350
[142]	validation_0-auc:0.99376	validation_1-auc:0.99353
[143]	validation_0-auc:0.99378	validation_1-auc:0.99355
[144]	validation_0-auc:0.99379	validation_1-auc:0.99356
[145]	validation_0-auc:0.99379	validation_1-auc:0.99357
[146]	validation_0-auc:0.99381	validation_1-auc:0.99358
[147]	validation_0-auc:0.99383	validation_1-auc:0.99360
[148]	validation_0-auc:0.99386	validation_1-auc:0.99363
[149]	validation_0-auc:0.99390	validation_1-auc:0.99367
[150]	validation_0-auc:0.99392	validation_1-auc:0.99369
[151]	validation_0-auc:0.99393	validation_1-auc:0.99370
[152]	validation_0-auc:0.99397	validation_1-auc:0.99374
[153]	validation_0-auc:0.99398	validation_1-auc:0.99374
[154]	validation_0-auc:0.99402	validation_1-auc:0.99378
[155]	validation_0-auc:0.99403	validation_1-auc:0.99379
[156]	validation_0-auc:0.99405	validation_1-auc:

[286]	validation_0-auc:0.99602	validation_1-auc:0.99567
[287]	validation_0-auc:0.99603	validation_1-auc:0.99567
[288]	validation_0-auc:0.99603	validation_1-auc:0.99567
[289]	validation_0-auc:0.99603	validation_1-auc:0.99567
[290]	validation_0-auc:0.99604	validation_1-auc:0.99568
[291]	validation_0-auc:0.99604	validation_1-auc:0.99568
[292]	validation_0-auc:0.99605	validation_1-auc:0.99568
[293]	validation_0-auc:0.99605	validation_1-auc:0.99569
[294]	validation_0-auc:0.99606	validation_1-auc:0.99569
[295]	validation_0-auc:0.99606	validation_1-auc:0.99569
[296]	validation_0-auc:0.99607	validation_1-auc:0.99570
[297]	validation_0-auc:0.99608	validation_1-auc:0.99570
[298]	validation_0-auc:0.99608	validation_1-auc:0.99571
[299]	validation_0-auc:0.99609	validation_1-auc:0.99571
[300]	validation_0-auc:0.99609	validation_1-auc:0.99571
[301]	validation_0-auc:0.99609	validation_1-auc:0.99572
[302]	validation_0-auc:0.99610	validation_1-auc:0.99572
[303]	validation_0-auc:0.99610	validation_1-auc:

[433]	validation_0-auc:0.99679	validation_1-auc:0.99630
[434]	validation_0-auc:0.99679	validation_1-auc:0.99630
[435]	validation_0-auc:0.99679	validation_1-auc:0.99630
[436]	validation_0-auc:0.99680	validation_1-auc:0.99630
[437]	validation_0-auc:0.99681	validation_1-auc:0.99632
[438]	validation_0-auc:0.99681	validation_1-auc:0.99632
[439]	validation_0-auc:0.99682	validation_1-auc:0.99632
[440]	validation_0-auc:0.99682	validation_1-auc:0.99633
[441]	validation_0-auc:0.99682	validation_1-auc:0.99633
[442]	validation_0-auc:0.99683	validation_1-auc:0.99633
[443]	validation_0-auc:0.99684	validation_1-auc:0.99633
[444]	validation_0-auc:0.99684	validation_1-auc:0.99634
[445]	validation_0-auc:0.99684	validation_1-auc:0.99634
[446]	validation_0-auc:0.99685	validation_1-auc:0.99635
[447]	validation_0-auc:0.99686	validation_1-auc:0.99635
[448]	validation_0-auc:0.99686	validation_1-auc:0.99635
[449]	validation_0-auc:0.99686	validation_1-auc:0.99635
[450]	validation_0-auc:0.99687	validation_1-auc:

[580]	validation_0-auc:0.99725	validation_1-auc:0.99661
[581]	validation_0-auc:0.99725	validation_1-auc:0.99661
[582]	validation_0-auc:0.99725	validation_1-auc:0.99661
[583]	validation_0-auc:0.99726	validation_1-auc:0.99662
[584]	validation_0-auc:0.99726	validation_1-auc:0.99662
[585]	validation_0-auc:0.99726	validation_1-auc:0.99662
[586]	validation_0-auc:0.99727	validation_1-auc:0.99662
[587]	validation_0-auc:0.99727	validation_1-auc:0.99662
[588]	validation_0-auc:0.99727	validation_1-auc:0.99662
[589]	validation_0-auc:0.99727	validation_1-auc:0.99662
[590]	validation_0-auc:0.99727	validation_1-auc:0.99662
[591]	validation_0-auc:0.99728	validation_1-auc:0.99662
[592]	validation_0-auc:0.99728	validation_1-auc:0.99662
[593]	validation_0-auc:0.99728	validation_1-auc:0.99663
[594]	validation_0-auc:0.99728	validation_1-auc:0.99663
[595]	validation_0-auc:0.99729	validation_1-auc:0.99663
[596]	validation_0-auc:0.99729	validation_1-auc:0.99663
[597]	validation_0-auc:0.99729	validation_1-auc:

...

[875]	validation_0-auc:0.99791	validation_1-auc:0.99688
[876]	validation_0-auc:0.99791	validation_1-auc:0.99688
[877]	validation_0-auc:0.99791	validation_1-auc:0.99688
[878]	validation_0-auc:0.99791	validation_1-auc:0.99688
[879]	validation_0-auc:0.99791	validation_1-auc:0.99688
[880]	validation_0-auc:0.99791	validation_1-auc:0.99688
[881]	validation_0-auc:0.99791	validation_1-auc:0.99688
[882]	validation_0-auc:0.99791	validation_1-auc:0.99688
[883]	validation_0-auc:0.99791	validation_1-auc:0.99688
[884]	validation_0-auc:0.99792	validation_1-auc:0.99689
[885]	validation_0-auc:0.99792	validation_1-auc:0.99689
[886]	validation_0-auc:0.99792	validation_1-auc:0.99689
[887]	validation_0-auc:0.99792	validation_1-auc:0.99689
[888]	validation_0-auc:0.99792	validation_1-auc:0.99689
[889]	validation_0-auc:0.99793	validation_1-auc:0.99689
[890]	validation_0-auc:0.99793	validation_1-auc:0.99689
[891]	validation_0-auc:0.99793	validation_1-auc:0.99689
[892]	validation_0-auc:0.99793	validation_1-auc:

[1021]	validation_0-auc:0.99809	validation_1-auc:0.99696
[1022]	validation_0-auc:0.99809	validation_1-auc:0.99696
[1023]	validation_0-auc:0.99810	validation_1-auc:0.99696
[1024]	validation_0-auc:0.99810	validation_1-auc:0.99696
[1025]	validation_0-auc:0.99810	validation_1-auc:0.99696
[1026]	validation_0-auc:0.99810	validation_1-auc:0.99696
[1027]	validation_0-auc:0.99810	validation_1-auc:0.99696
[1028]	validation_0-auc:0.99810	validation_1-auc:0.99696
[1029]	validation_0-auc:0.99810	validation_1-auc:0.99696
[1030]	validation_0-auc:0.99810	validation_1-auc:0.99696
[1031]	validation_0-auc:0.99811	validation_1-auc:0.99697
[1032]	validation_0-auc:0.99811	validation_1-auc:0.99697
[1033]	validation_0-auc:0.99811	validation_1-auc:0.99697
[1034]	validation_0-auc:0.99811	validation_1-auc:0.99697
[1035]	validation_0-auc:0.99811	validation_1-auc:0.99697
[1036]	validation_0-auc:0.99812	validation_1-auc:0.99697
[1037]	validation_0-auc:0.99812	validation_1-auc:0.99697
[1038]	validation_0-auc:0.99812

[1165]	validation_0-auc:0.99825	validation_1-auc:0.99701
[1166]	validation_0-auc:0.99825	validation_1-auc:0.99701
[1167]	validation_0-auc:0.99825	validation_1-auc:0.99701
[1168]	validation_0-auc:0.99825	validation_1-auc:0.99701
[1169]	validation_0-auc:0.99825	validation_1-auc:0.99701
[1170]	validation_0-auc:0.99825	validation_1-auc:0.99701
[1171]	validation_0-auc:0.99825	validation_1-auc:0.99701
[1172]	validation_0-auc:0.99825	validation_1-auc:0.99701
[1173]	validation_0-auc:0.99825	validation_1-auc:0.99701
[1174]	validation_0-auc:0.99825	validation_1-auc:0.99701
[1175]	validation_0-auc:0.99825	validation_1-auc:0.99701
[1176]	validation_0-auc:0.99825	validation_1-auc:0.99701
[1177]	validation_0-auc:0.99826	validation_1-auc:0.99701
[1178]	validation_0-auc:0.99826	validation_1-auc:0.99701
[1179]	validation_0-auc:0.99826	validation_1-auc:0.99701
[1180]	validation_0-auc:0.99826	validation_1-auc:0.99701
[1181]	validation_0-auc:0.99826	validation_1-auc:0.99701
[1182]	validation_0-auc:0.99826

[1309]	validation_0-auc:0.99836	validation_1-auc:0.99705
[1310]	validation_0-auc:0.99837	validation_1-auc:0.99705
[1311]	validation_0-auc:0.99837	validation_1-auc:0.99705
[1312]	validation_0-auc:0.99837	validation_1-auc:0.99705
[1313]	validation_0-auc:0.99837	validation_1-auc:0.99705
[1314]	validation_0-auc:0.99837	validation_1-auc:0.99705
[1315]	validation_0-auc:0.99837	validation_1-auc:0.99705
[1316]	validation_0-auc:0.99838	validation_1-auc:0.99706
[1317]	validation_0-auc:0.99838	validation_1-auc:0.99706
[1318]	validation_0-auc:0.99838	validation_1-auc:0.99705
[1319]	validation_0-auc:0.99838	validation_1-auc:0.99705
[1320]	validation_0-auc:0.99838	validation_1-auc:0.99705
[1321]	validation_0-auc:0.99838	validation_1-auc:0.99705
[1322]	validation_0-auc:0.99838	validation_1-auc:0.99706
[1323]	validation_0-auc:0.99838	validation_1-auc:0.99706
[1324]	validation_0-auc:0.99838	validation_1-auc:0.99706
[1325]	validation_0-auc:0.99838	validation_1-auc:0.99706
[1326]	validation_0-auc:0.99838

[1453]	validation_0-auc:0.99849	validation_1-auc:0.99710
[1454]	validation_0-auc:0.99849	validation_1-auc:0.99710
[1455]	validation_0-auc:0.99850	validation_1-auc:0.99710
[1456]	validation_0-auc:0.99850	validation_1-auc:0.99710
[1457]	validation_0-auc:0.99850	validation_1-auc:0.99710
[1458]	validation_0-auc:0.99850	validation_1-auc:0.99710
[1459]	validation_0-auc:0.99850	validation_1-auc:0.99710
[1460]	validation_0-auc:0.99850	validation_1-auc:0.99710
[1461]	validation_0-auc:0.99850	validation_1-auc:0.99710
[1462]	validation_0-auc:0.99850	validation_1-auc:0.99710
[1463]	validation_0-auc:0.99850	validation_1-auc:0.99710
[1464]	validation_0-auc:0.99850	validation_1-auc:0.99710
[1465]	validation_0-auc:0.99850	validation_1-auc:0.99710
[1466]	validation_0-auc:0.99850	validation_1-auc:0.99710
[1467]	validation_0-auc:0.99850	validation_1-auc:0.99710


In [ ]:
#阈值大概在0.2-0.4之间 本题对召回率较敏感，可适当降低一下阈值
thre = 0.25

In [ ]:
#生成提交文件
sub = pd.DataFrame()
sub['hit_id']=testpd['hit_id']
sub['flag_pred'] = y_pp_xgb
sub['event_id'] = testpd['event_id']
sub['flag_pred'] = sub['flag_pred'].apply(lambda x: 1 if x >= thre else 0)
sub.to_csv(os.path.join(pathf, "subsample.csv").format(sub['flag_pred'].mean()),index=False)